In [1]:
from config import setup
setup()

# Projet AP : Late Evacuation Planning Problem

## INTRO
TODO :
- présenter le contexte

 ## Partie 1 :  Generator 
 TODO : A déplacer après la partie modélisation
 Generator est une instance qui permet de générer aléatoirement un réseau de routes et de propager un simple model de feu. Les données générées prennent en compte des propagations de contraintes qu'il convient d'expliquer et de comprendre !
 
 ### 1.1 Install module
 Pour générer les données utilisées après par le projet, il faut utiliser le générateur du projet evacsim présent sur le git https://github.com/ehebrard/evacsim
 Il faut l'utiliser avec python 2 et avoir installé decorator et networkx.
 Note : il faut un dossier data dans le projet. C'est dans ce dossier que seront déposées les données générées.  
 
 ***
 
 Commande pour installer :  
 `python setup.py install --user`
 
 Les commandes pour générer :  
 `python generator.py --road test --printroad`  
 `python generator.py test --evacuation --printfire --seed 10`
 
 NOTE : avec la commande : `python generator.py --road test --evacuation --printfire --seed 10 --printevac`on peut voir la route d'évacuation  
 
 ***  
 
 Pour générer une nouvelle instance de feu, il suffit de changer le paramètre seed.
 
 ### 1.2 Modélisation données générées
 FORMAT :
 n m population_1 maximum_rate_1 duedate_1 ... population_n maximumrate_n duedate_n capacity_1 k_1 i_1_1 offset_i_1_1 ... i_k_1 offset_i_k_1 ... capacity_m k_m i_1_m offset_i_1_m ... i_k_m offset_i_k_m
 
- **n** is the number of evacuation nodes
- **m** is the number of relevant transit arcs 
- **Population_i** : population du noeud i a évacuer.
- **Maximum_rate_i** : taux max de personnes pouvant être évacuées en même temps
- **duedate_i** : Date à laquelle le noeud doit être évacué
- **capacity_y** is the capacity of transit arc y
- **k_y** is the number of population groups transiting by this arc
- **offset_i_x_y** is the date at which population group i_x_y reaches this arc if starting at time 0

EXEMPLE :   
- l1 : `10 8`  
10 zones à évacuer avec 8 arcs à utiliser pour évacuer les zones  
- l2 : `1677 70 66`   
sur le sommet 1, 1677 personnes à évacuer, par pacquets de 70, avant la date 66  
- l3 : `4161 70 40`  
- l4 : `3817 70 36`  
- l5 : `3745 70 92`  
- l6 : `1379 72 126`   
- l7 : `3359 71 115`  
- l8 : `893 72 120`  
- l9 : `463 72 54`  
- l10: `4368 212 86`  
- l11: `4987 70 44`  
Sommet 10  
- l12: `74 2 3 20 6 8`  
Arc 1, capacité de 74 personnes, Nombre de Groupes transitant par cet arc(2), puis lire par tuple de 2 (3,20) : 3, num du groupe concerné, 20, date au plus tôt à laquelle la population peut partir si il part à t=0. Cela veut donc dire que si le groupe 3 part a t=4, il pourra traverser l'arc 1 à partir de t=24.(6,8).  
- l13: `74 4 2 1 5 2 7 4 9 2`  
arc2, capacité74 personnes, 4 groupes possibles puis (2,1), (5,2), (7,4), (9,2)  
- l13: `206 8 0 13 1 15 2 16 4 14 5 17 7 19 8 9 9 17`  
- l14: `72 2 0 2 1 4`  
- l15: `203 10 0 33 1 35 2 36 3 40 4 34 5 37 6 28 7 39 8 29 9 37`  
- l16: `72 5 2 4 4 2 5 5 7 7 9 5`  
- l17: `72 2 7 3 9 1`  
- l18: `138 7 0 8 1 10 2 11 4 9 5 12 7 14 9 12`  
Arc 8 capacité de 138 personnes, ...    


**NOTE IMPORTANTE ** : les couleurs des arcs correspondes à la capacité de la route. 


## Partie 2 :  GEO-SAFE
TODO : A reformuler pour l'intro  
Présentation d'après l'artique A study of Evacuation Planning for Wildfires de Christian Artigues, Emmanuel Hebread, Yanninck Pencole, Andreas Schutt et Peter Stuckey. (dans le dossier /docs du git)

### 2.1 Présentation
Le but de ce projet est de pouvoir **assister la prise de décision des autorités lors de l'évacuation à grande échelle (de la population) pendant un feu de forêts**. Pour cela, il faut modéliser le problème pour vérifier ensuite que  l'évacuation de la population pendant un feu est bien réalisable. 
Pour cela, on utilise des outils d'optimisation.

La population se trouve sur des noeuds à évacuer. Dans la "vraie vie", il y a 3 catégories de personnes à évacuer : les personnes qui partent tôt, celles qui se réfugie dans un lieu et celles qui restent "se battre". Les derniers sont les personnes qui concernent l'objet de la modélisation, ce sont des personnes qui partent tard.

Dans le cas de modélisation d'evacuation lors d'innondation, les méthodes utilisées étaient malléables pour représenter l'évacuation de la population. Il y avait des contraintes cumulatives par segments de route. 
La propagation des feu est beaucoup moins prédictible que celle des innondations. En effet, cela n'est pas fixe selon la topologie, mais dépend plus de la présence de combustible et de la vitesse et direction du vent. Donc la méthode d'évacutaion doit être plus robuste pour pouvoir prédire différents scénarios. 

Il faut éviter le risque de congestion et donc, pour cela, on peut **retarder le départ de la population**. Il est aussi possible **d'influer sur le taux d'évacuation** (moduler la méthode pour déclancher l'alarme).

En plus de minimiser le temps d'évaciation de la population, le but est de maximiser le minimum "safety margin" spatial et temporel pondéré par la population, sur chaque segment de route. Il faut donc être le plus loin et le plus rapidement possible du feu.

L'équipe GeoSafe a introduit une heuristique et un propagateur de contraintes de fluw global.


## Partie 3 : Evacuation Planning Problem
Notre équipe (Duc, Adrian et Anaïs) a choisi de modéliser le problème de façon à le résoudre en **programmation par contraintes** avec CP Optimizer.

Il faut que l'on trouve la date à laquelle on commence l'évacuation pour chaque noeud, ainsi que le taux d'évacuation. Le taux d'évacuation est en fait une modélisation du niveau de ressources utilisées par les autorités pour évacuer les gens.  

### 3.1 Hypothèses
Quelques hypothèses posées par l'équipe GeoSafe :
- Les authorités ont déjà identifié des routes accessibles et des points de refuges en sécurité. Chaque sommet à évacuer a donc une seule route d'évacuation. Il y a 1 seul et même point de refuge à atteindre par toute la population a évacuer. Il se peut qu'il y ait une congestion sur un des segment de route partagé par plusieurs points.
- Les autorités ont déja estimé le nombre d'habitant à évacuer (la population de ceux qui partent tard)
- la modélisation du feu donne une deadline pour chaque segment (date à laquelle le feu arrive).
- Une fois que l'évacuation d'un point a commencé, le process ne peut pas être interrompu. Les personnes en cours d'évacuation ne s'arrêtent jamais
- C'est non-préemptif.
- le flux d'évacuation reste constant durant tout le processus d'évacuation.

### 3.2 Modélisation 
Un argre $G= (\varepsilon \cup T \cup \{r\}, A)$ qui représente les routes depuis les sommets à évacuer $\varepsilon$ vers le refuge $r$ en passant par des noeuds de transit $T$

- sur chaque noeud $v\in\varepsilon$, il y a $W_{v}$ personnes à évacuer. Cela correspond à **population_i** du .evac.
- Chaque arc alant de $u$ à $u'$, de longueur $l_{uu'}$ a une capacité $q_{u}$. Cela corespond à **capacity_y** du .evac.
- $H=[0,h]$ représente l'interval de temps pour l'évacuation.
Il faut associer chaque $v$ à un réel $s_{v}$ représentant le **retard du préavis d'évacuation** ainsi qu'a une courbe de réponse $\phi_{v}$ décrivant le **flow d'évacuation** d'un noeud $v$ a un taux $\phi_{v}(t)$. avant $s_{v}$ le taux $\phi_{v}$ est à 0.
$$\int_{0}^{H}\phi_{v}(t)dt=w_{v} $$
Le flow $\phi_u$ pour n'importe quel arc $u-v$ à n'importe quel moment noeud : 
$$\phi_u(t)=\sum_{v\in descendants(u)} \phi_{v}(t-l_{uv}-s_{v})$$  

On peut aussi le réécrire :
$$\phi_u(t)=\sum_{V\in descendants(u),\: s_v+l_{uv}\le t \lt s_v+l_{uv}+\frac{w_v}{h_v}} h_v$$  

Ici, $\phi_v$ est une courbe de réponse simple. Le flux sortant $\phi_v$ est une variable de décision continue et qui reste constante penant le process d'évacuation.
On note aussi :
$$ \phi_v = h_v\: during\: [s_v,e_v]\: with\: e_v=s_v + \frac{w_v}{h_v}$$


- Il y a une tache pour chaque noeud à évacuer. Pour chaque tâche, on a un **taux constant d'évacuation** $\phi_v \le q_{v}$ et une **heure de début d'évacuation** $s_{v}\in [0, H-\frac{w_v}{h_v}]$

**CONTRAINTES** :   
Il n'y a qu'un seul type de contraintes pour éviter les bouchons (donc éviter d'avoir un flux plus grand que la capacité d'un arc) : $\phi_v \le q_{v}$

**OBJECTIF** :  
Chaque noeud de transit a une date au plus tard (due date) $d_u$, date à partir de laquelle la route n'est plus en sécurité (à cause du feu). L'objectif est donc de minimiser l'écart de temps maximal entre lequel la population quitte $u$ et la due date $d_u$.
$$\min \max_{u\in T,v\in descendants(u)} s_v + \frac{h_v}{w_w} + l_{uv} - d_u$$

Cette fonction objectif se simplifie en 
$$\min \max_{v\in\varepsilon} s_v + \frac{h_v}{w_w} - d_v$$
avec $d_v=\min_{u\in T}\{d_u-l_{uv}\}$

**OBSERVATION**:   
Pour chaque noeud de transit $u$ et $u'$, avec $u'$ déscendant de $u$ un bouchon sur $u'$ entraine un bouchon sur $u$. Donc, il ne faut checker l'occurance de bouchons uniquement si la capacité de $u$ est supérieur à la capacité de sons ascendant $v$ ( $\forall v \in p'(u),q_u \gt q_v$ )  

**SIMPLIFICATION**:     
Sur un chemin de $u$ vers $u'$ sans branches, on garde la capacité de l'arc minimum. 
Par exemple, avec c'<c, on garde donc la capacité c' :  
            S5  `  
```
S1 ___ S2 ___ S3 ___ S4  
     c     c'  |    
               S5  
```
   
```
S1 ___ S2 ___ S3 ___ S4  
     c'     c'  |    
               S5  
```


### 3.3 Utilisation des contraintes cumulatives
L'approche de base se fait avec des contraintes cumulatives simples.  
**NOTATION** :   
Pour une variable $x$, $\bar{x}$ signifie la plus grande valeur de x et $\underline{x}$, la plus petite valeur de x. 

Si on prend une selection de **tâches** $J$ avec une **heure de départ** $s_{i} \in [\underline{s_i}, \bar{s_i}]$, avec une **durée de process** $p_i \in [\underline{p_i},\bar{p_i}]$, une **hauteur** $h_i \in [\underline{h_i}, \bar{h_i}]$ et une **resource** $r$ avec une **capacité constante** $q_r$, on défini la contrainte cumulative comme :
$$ \sum_{i\in J|s_i\le t\le s_i+p_i} h_i \le q_r, \forall t\in H$$

Donc pour modéliser le problème, il suffit d'associer une tâche $v$ à chaque noeud à évacuer ($v \in \varepsilon$), avec une hauteur $h_v \in ]0, q_v]$, avec une heure de départ $ s_v \in [0, H-\frac{w_v}{q_v}]$, une heure de "completion" $e_v \in [\frac{w_v}{q_v}, H]$. Puis il faut dupliquer et transformer cette tâche pour chaque arc critique de transit jusqu'au point de refuge. On note $i_{uv}$ la duplication du noeud $v$ sur l'arc critique $u$.

On a donc :
$$cumulative((s_{i_{uv}}, e_{i_{uv}} - s_{i_{uv}}, h_{i_{uv}})_{v\in L(u)}, q_u) \forall u \in T $$
$$w_v=h_v(e_v-s_v) \forall v \in \varepsilon$$
$$s_{i_{uv}} = s_v +l_{uv} \forall u\in T, \forall v \in L(u)$$
$$e_{i_{uv}} = e_v +l_{uv} \forall u\in T, \forall v \in L(u)$$
$$h_{i_{uv}} = h_v \forall u\in T, \forall v \in L(u)$$

Le problème avec les contraintes cumulatives "classiques" c'est qu'elles ne considères que les bornes inférieurs des domaines de solutions possibles. (*cf.exemple p7 de l'article*). Elles ne permettent pas de raisonner avec l'énergie totale de chaque tâche.


### 3.4 La contrainte cumulative énergétique
Le but de cette nouvelle contrainte est solliciter le fait que le produit $(durée * taux)$ soit constant. Pour cela il faut une contrainte un peu plus globale. 
La contrainte **energetic_cumulative**($(s_i,e_i,h_i,w_v)_{i\in J}, q_r$) se traduit comme :
$$ \sum_{i\in J |s_i\le t\le e_i} h_i \le q_r \forall t\in H$$
$$w_i=h_i(e_i-s_i) \forall i \in J$$
Cette contrainte est NP-complet puisque le cas particulier où la hauteur $h_i$ est fixée est une contrainte cumulative.
Cependant, comme les tâches sont malléables, le problème devient plus simple sous certaines hypothèses. 

En particulier :
- on considère que l'on n'a pas les valeurs minimum des hauteurs $h_i$ des tâches i, que l'on note **energetic_cumulative** \ $\{ h-\}$.

On peut donc définir 3 relaxations :
- "$r$" représente la release date pour laquelle la relaxation signifie que $s_i-=0$ pour chaque tache $i\in J$
- "$d$" représente la date butoire pour laquelle la relaxation implique $e_i+=H$ pour chaque tâche $i\in J$
- "$h+$" qui représente la hauteur maximale por laquelle la relaxation implique $h_i+=q_r$ pour chaque tâche $i \in J$

On peut modéliser la contrainte cumulative énergétique pour slaquelle les contraintes $S \in \{h-, h+, r,d\}$ sont relaxée.

**THEOREME 1** **energetic_cumulative**  $\{h-,x,y\}\: is\: in\: P\: for\: any\: x \ne y \in \{r,d,\bar{h}\}\: scénarios$

L'algorithme 1 permet de plannifier des tâches qui commencent à 0 et qui finnissent au plus tard à $h_i \le h_i+$. La contrainte est donc satisfaisable si et seulement si l'heure de completion la plus tard est inférieure à H. 

**THEOREM 2** **Energetic_cumulative** $\{h-,x,y\}\: is\: NP-Complete\: for\: any\: x \in \{r,d,\bar{h}\}$

### 3.5 Contraintes de Flow Global
 Si on veut avoir un taux d'activité qui varie avec le temps, on peut utiliser un propagateur de flux adapté à la contrainte **energetic-cumulative**.
 Ce propagateur fonctionne comme ci-après :
 - Construit un réseau de flow relaxé, appelé $f(D)$ 
 - propage en utilisant le domaine courrant $D$.
 
**THEOREM 3** $any \: solution \: to$ **Energetic_cumulative** $\: given\: a\: current\: domain \: D,\: is\: extensible\: to\: a\: solution\: of\: the\: flow\: network\: f(D)$
 
 Pour plus d'info, regarder la partie 5 de l'article. 

### 3.6 Heuristique pour la borne supérieur
Heuristique de compression pour trouver une borne supérieur initiale.  
On suppose que le domaine du taux initial d'évacuation est $D(h_v)=[1,q_v]$ pour chaque noeud d'évacuation.
On suppose aussi que la planification des tâches d'évacuation commençantt à $t=0$ avec le taux minimum d'évacuation permet de trouver une solution fesable, avec un coût élevé. 

Commençant avec cette soution, $(\forall v \in \varepsilon, s_v :=0, e_v=w_v, h_v:=1)$, les heures de départ minimales et maximales sont construite pour chaque noeud de transit. De même pour interval correspondant pour la contrainte de flow global.

A partir de cette solution, un process iteratif est lancé. A chaque itération, la tâche critique est identifiée. Puis, sa durée est diminuée et sa largeur est augmentée en conséquence jusqu'à ce qu'on ne puisse plus augmenté/diminuer la largeur/durée sans dépasser la capacité de l'arc ou que la tâce ne devienne plus critiqeu et qu'une autre tâche le devienne.  Dans le cas où une nouvelle tâche devient critique, le process itératif se relance sur ce nouvel arc.   

L'heuristique peret de donner une solution initiale au solver. 

## Résultats de l'équipe GeoSafe

TODO

# Partie 4 : PISTES pour modéliser, résoudre

CP Optimizer vient avec CPLEX et OPL in IBM ILOG CPLEX Optimization Studio

lien pour la contrainte cumulative (pulse, ...)
https://www.ibm.com/support/knowledgecenter/en/SSSA5P_12.6.3/ilog.odms.cplex.help/refcppcplex/html/cumul_functions.html

TODO : Faire avec le tuto !!

 ---------------------------------------------------------
 ## 4.1 Generator 
 Pour générer les données utilisées après par le projet, il faut utiliser le générateur du projet evacsim
 

In [4]:
from docplex.cp.model import CpoModel
from docplex.cp.model import *
import numpy as np

In [5]:
# CONFIG 
from config_duc import setup
setup()

## 4.2 MODELISATION 

In [39]:
def geosafe_model(l, q, eps, r, W, H, d,chemin):
    '''       
    Int  Matrix l: length path from i to j = G.l[i,j]. No path ij <--> G.l[i,j] = 0
    Int  Array  q: capacity of node i <--> G.q[i]
    Bool Array  eps: evacuation node 
    Bool Array  r: safe node
    Int  Array  W: initial population at node i <--> W[i]
    Int         H : Time span
    Int  Array  d: deadline to leave the node
    '''
    
    nb_node = q.shape[0] # ou len(q)
    nodes = np.arange(nb_node) # id nodes, i.e [1,2,3,4, ...]
    total_population = np.sum(W)
    
    t = !eps & !r # transition node

    mdl = CpoModel(name='geosafe')
    
    # == Output ==
    # starting date
    s = np.array( mdl.integer_var_list(nb_node, min=0, max = H, name="s") ) # TODO : affiner max =H-w(v)/q(v)
    
    # evacuation rate aka. height of package
    h = np.array( mdl.integer_var_list(nb_node, min=0, max = 300, name="h") ) # T0DO : affiner max =q[v]

    # == Intermediate ==
    # node flow of population for every node during [0,H]. It enables us to have phi for every transit node u.
    phi = np.matrix([
            [ 
                mdl.integer_var(name='phi[%d,%d]'%(i,j)) for j in range(H) 
            ] for i in range(nb_node)
        ])
    
    # ending date (leaving time) of node
    e = np.array( mdl.integer_var_list(nb_node, min=0, max = H, name= "e") ) # TODO : min c'est w[v]/q[v] 
    
    # == Constraints ==
    
    # La date de départ des noeuds    
    for v in range(nb_node):
        if eps[v]:
            mdl.add(
                e[v] == s[v] + W[v]/h[v]
            )
    # evacuate everyone in evacuation node. On s'assure que tous les noeuds eps ont le temps d'être évacués
    for v in nodes[eps]:
        mdl.add( h[v]*H >= W[v] ) 
    
    # Flow at a node u = sum from all of its leaves node epsilon
#     mdl.add_constraint(
#         phi[u, t] == np.sum( phi[u, t - l[u,v] - s[eps]] ) \
#             for t in range(H)                                  \
#             for v in np.where(eps)                           \
#             for u in np.where(t)                             
#     )

    #  CONTRAINTE CUMULATIVE
    for v in nodes[eps]: # pour chaque resource on associe une tâche 
        # que l'on duplique ensuite sur chaque arc critique
        #wv=h[v](e[v]-s[v])
        phiarc=step_at(0, 0)
        for u in chemin[v]:  # Astuce pour ajouter une contrainte uniquement si u est dans le chemin emprunté par V
            siuv = s[v]+l[v,u]
            eiuv = e[v]+ l[v,u]
            phiarc += mdl.pulse((siuv, eiuv-siuv), h[v])
            
            mdl.add( phiarc <= q[u])
    
    # Flow at a node does not excess capacity of arc
    # TODO : deleted to see if cumulative constraint is working
    #for t in range(H):
     #   for u in nodes:
     #       mdl.add(
     #           phi[u,t] <= q[u]
     #       )
    
    # Objective
    mdl.add(
        minimize(np.max(s[eps] + h[eps]/W[eps] - d[eps]))
    )
    
    return mdl

## 4.3 UNIT TEST :

- Test input 


In [21]:
import networkx as nx

In [22]:
G = nx.Graph()

global_d = 250 # TODO: delete this one, just for debug

# TODO: Maybe we should use eps = W > 0, so that do not need to use 'eps' attribute?
nodes_state = [
    (1, {'eps': True,  'r': False, 'W': 5, 'd': global_d}),
    (2, {'eps': True,  'r': False, 'W': 5, 'd': global_d}),
    (3, {'eps': True,  'r': False, 'W': 5, 'd': global_d}),
    (4, {'eps': False, 'r': False, 'W': 0, 'd': global_d}),
    (5, {'eps': False, 'r': False, 'W': 0, 'd': global_d}),
    (6, {'eps': False, 'r': True,  'W': 0, 'd': global_d})
]

edges_state = [
    (1, 4, {'l' : 4}),
    (2, 4, {'l' : 3}),
    (4, 5, {'l' : 7}),
    (3, 5, {'l' : 3}),
    (5, 6, {'l' : 10})
]
chemin = [
    [1,3,5],
    [2,3,5],
    [4,5]
]

G.add_nodes_from(nodes_state)
G.add_edges_from(edges_state)

In [23]:
def _node_attribute_(G, attribute='eps'):
    '''
    Author: Duc Hau :D
    Return a numpy boolean array of attribute
    '''
    values_tmp = nx.get_node_attributes(G, attribute).values()
    return np.array(list(values_tmp))

In [24]:
print(_node_attribute_(G, attribute='W'))

[5 5 5 0 0 0]


- Init Model

In [25]:
# Path length
l = nx.to_numpy_matrix(G, weight='l')

# Node capacities (shouldn't it be the path??)
q = np.array([20,20,20,20,20,20])

# If the node is in EPSILON (evacuating node)
eps = _node_attribute_(G, 'eps')

# If the node is in the ROOT (safe node)
r = _node_attribute_(G, 'r')

# Initial population
W = _node_attribute_(G, 'W')

# Time span
H = 200

# Deadline for evacutation
d = _node_attribute_(G, 'd')

# Solver configuration
ctx = {}

In [40]:
mdl = geosafe_model(l, q, eps, r, W, H, d, chemin)

AssertionError: Argument 'interval' should be an interval variable or a fixed interval expressed as a tuple of integers

2
3
5


- Solve model

In [ ]:
mdl.solve()
sol = mdl.solve(Presolve='On', Workers='Auto')
print(sol.get_solver_log())

print('\n\n Result:')
if sol.is_solution():
    sol.print_solution()
else:
    print('Problem does not have solution')
    sol.print_solution()